In [19]:
import os
import re

def check_folders(root_folder):
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)

        # 폴더인지 확인
        if not os.path.isdir(folder_path):
            continue

        # 파일 리스트 (파일만)
        files = [
            f for f in os.listdir(folder_path)
            if os.path.isfile(os.path.join(folder_path, f))
        ]

        file_count = len(files)

        if file_count == 0:
            print(f"[{folder_name}] 파일 없음")
            continue

        # ✅ gif 파일 존재 여부 확인
        has_gif = any(f.lower().endswith(".gif") for f in files)
        if has_gif:
            print(f"[{folder_name}] ⚠️ GIF 파일 포함됨")

        # 파일 정렬 (이름 기준 정렬)
        files.sort()

        last_file = files[-1]

        # img_숫자 패턴 찾기
        match = re.search(r"img_(\d+)", last_file)

        if not match:
            print(f"[{folder_name}] 마지막 파일 형식이 올바르지 않음: {last_file}")
            continue

        last_number = int(match.group(1))

        if last_number != file_count:
            print(f"[{folder_name}] ❌ 불일치 (파일 개수: {file_count}, 마지막 번호: {last_number})")


print('완료')

# 사용 예시
root_folder = r"Death_Note"
check_folders(root_folder)

완료


In [ ]:
# 깨진 이미지 찾기

In [23]:
import os
import cv2
import numpy as np

# ==============================
# 설정
# ==============================
root_folder = r"DragonBall"
reference_image_path = r"last_image.jpg"
similarity_threshold = 0.95  # 1에 가까울수록 엄격

# ==============================
# 이미지 유사도 계산 함수
# ==============================
def image_similarity(img1, img2):
    # 크기 통일
    size = (300, 300)
    img1 = cv2.resize(img1, size)
    img2 = cv2.resize(img2, size)

    # grayscale 변환
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # 차이 계산
    diff = cv2.absdiff(img1, img2)
    score = 1 - (np.mean(diff) / 255)

    return score


# ==============================
# 기준 이미지 로드
# ==============================
ref_img = cv2.imread(reference_image_path)

if ref_img is None:
    raise Exception("기준 이미지 로드 실패")

print("검사 시작...\n")

# ==============================
# 폴더 순회
# ==============================
for root, dirs, files in os.walk(root_folder):
    image_files = [
        f for f in files
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    if not image_files:
        continue

    # 파일명 정렬 (마지막 이미지 찾기 위함)
    image_files.sort()
    last_image_path = os.path.join(root, image_files[-1])

    img = cv2.imread(last_image_path)
    if img is None:
        continue

    score = image_similarity(ref_img, img)

    if score < similarity_threshold:
        print(f"[불일치] 폴더: {root}")
        print(f"  마지막 이미지: {image_files[-1]}")
        print(f"  유사도: {score:.4f}\n")

print("검사 완료")

검사 시작...

[불일치] 폴더: DragonBall\215
  마지막 이미지: img_008.jpg
  유사도: 0.5118

[불일치] 폴더: DragonBall\262
  마지막 이미지: img_002.jpg
  유사도: 0.6049

검사 완료
